# A MultiClass Neural Network #

Let's try coding a Multiclass neural network from scratch

In [18]:
# import useful librairies
import numpy as np

In [24]:
# Let's make a multilabel dataset

# Initialize randomness
np.random.seed(42)

# Our dataset is made of coded numbers
zero = np.array([[1],[1],[1],[0],[1],[1],[1]])
one = np.array([[0],[0],[0],[0],[0],[1],[1]])
two = np.array([[0],[1],[1],[1],[1],[1],[0]])
three = np.array([[0],[0],[1],[1],[1],[1],[1]])
four = np.array([[1],[0],[0],[1],[0],[1],[1]])
five = np.array([[1],[0],[1],[1],[1],[0],[1]])
six = np.array([[1],[1],[1],[1],[1],[0],[1]])
seven = np.array([[0],[0],[1],[0],[0],[1],[1]])
eight = np.array([[1],[1],[1],[1],[1],[1],[1]])
nine = np.array([[1],[0],[1],[1],[1],[1],[1]])

# A function to decode our number
def decode_num(x):
    r = x
    r = np.insert(r, 0, [0])
    r = np.insert(r, -2, [0])
    r = r.reshape(9,1)
    r = np.append(r[:3], np.append(r[3:6], r[6:], axis = 1), axis = 1)
    stringa = ''
    stringb = ''
    stringc = ''
    for i,y in enumerate(r):
        for j,v in enumerate(y):
            if i == 0:
                if v == 1 and j == 1:
                    stringa = stringa + '_'
                else:
                    stringa = stringa + ' '
            if i == 1:
                if v == 1 and j == 1:
                    stringb = stringb + '_'
                elif v == 1:
                    stringb = stringb + '|'
                else:
                    stringb = stringb + ' '
            if i == 2:
                if v == 1 and j == 1:
                    stringc = stringc + '_'
                elif v == 1:
                    stringc = stringc + '|'
                else:
                    stringc = stringc + ' '
    print(stringa)
    print(stringb)
    print(stringc)

# Making m examples from our coded numbers
m = 20
# Choosing randomly the numbers for m examples
pre_dataset = np.random.randint(0, 10, (1, m))

# Setting labels
Y = np.zeros(0)
for v in pre_dataset[0]:
    Y = np.append(Y, [0 for x in range(v)] +[1] + [0 for x in range(9-v)])
Y = Y.reshape(m,10)

# Setting inputs
numbers = [zero, one, two, three, four, five, six, seven, eight, nine]
X = np.zeros(0)
for v in pre_dataset[0]:
    X = np.append(X, [numbers[v]])
X = X.reshape(m,7)

# Let's check if everything works find
for i, v in enumerate(Y):
    decode_num(X[i])
    print('label : ' , np.where(v == 1)[0])

 _ 
|_ 
|_|
label :  [6]
 _ 
 _|
 _|
label :  [3]
 _ 
  |
  |
label :  [7]
   
|_|
  |
label :  [4]
 _ 
|_ 
|_|
label :  [6]
 _ 
|_|
 _|
label :  [9]
 _ 
 _|
|_ 
label :  [2]
 _ 
|_ 
|_|
label :  [6]
 _ 
  |
  |
label :  [7]
   
|_|
  |
label :  [4]
 _ 
 _|
 _|
label :  [3]
 _ 
  |
  |
label :  [7]
 _ 
  |
  |
label :  [7]
 _ 
 _|
|_ 
label :  [2]
 _ 
|_ 
 _|
label :  [5]
   
|_|
  |
label :  [4]
   
  |
  |
label :  [1]
 _ 
  |
  |
label :  [7]
 _ 
|_ 
 _|
label :  [5]
   
  |
  |
label :  [1]


array([0., 0., 1., 0., 0., 1., 1.])